There are a total of n courses you have to take, labeled from 0 to n-1.

Some courses may have prerequisites, for example to take course 0 you have to first take course 1, which is expressed as a pair: [0,1]

Given the total number of courses and a list of prerequisite pairs, return the ordering of courses you should take to finish all courses.

There may be multiple correct orders, you just need to return one of them. If it is impossible to finish all courses, return an empty array.

Example 1:

Input: 2, [[1,0]] 
Output: [0,1]
Explanation: There are a total of 2 courses to take. To take course 1 you should have finished   
             course 0. So the correct course order is [0,1] .

Example 2:

Input: 4, [[1,0],[2,0],[3,1],[3,2]]
Output: [0,1,2,3] or [0,2,1,3]
Explanation: There are a total of 4 courses to take. To take course 3 you should have finished both     
             courses 1 and 2. Both courses 1 and 2 should be taken after you finished course 0. 
             So one correct course order is [0,1,2,3]. Another correct ordering is [0,2,1,3] .

Note:

    The input prerequisites is a graph represented by a list of edges, not adjacency matrices. Read more about how a graph is represented.
    You may assume that there are no duplicate edges in the input prerequisites.



# BFS - O(ve) runtime, O(v + e) space where v is the number of courses and e is the number of dependancies

In [1]:
from collections import defaultdict

class Solution(object):
    
    def findOrder(self, numCourses, prerequisites):
        """
        :type numCourses: int
        :type prerequisites: List[List[int]]
        :rtype: List[int]
        """
        if prerequisites == []:
            order = []
            for i in range(numCourses):
                order.append(i)
            return order
        
        traversal_graph = defaultdict(list)
        dependancy_graph = defaultdict(list)
        
        for dependancy in prerequisites:
            traversal_graph[dependancy[1]].append(dependancy[0])
            dependancy_graph[dependancy[0]].append(dependancy[1])
            
        roots = [x for x in traversal_graph.keys() if x not in dependancy_graph.keys()]
        
        node_set = set()
        for x in traversal_graph.keys():
            node_set.add(x) 
        for x in dependancy_graph.keys():
            node_set.add(x) 
        
        queue = []
        order = []
        for root in roots:
            queue.append(root)
        
        while queue:
            node = queue.pop(0)
            dep_check = 1
            for check_req in dependancy_graph[node]:
                if check_req not in order:
                    dep_check = 0
                    break
            if node not in order and dep_check == 1:              
                order.append(node)
            else:
                continue
            for next_node in traversal_graph[node]:
                queue.append(next_node)

        if len(order) == len(node_set):
            for i in range(numCourses):
                if i not in order:
                    order.append(i) 
            return list(order)
                
        
        return []       

# Using Node Indegree  - O(v + e) runtime, O(v + e) space where v is the number of courses and e is the number of dependancies 

In [4]:
from collections import defaultdict, deque
class Solution:

    def findOrder(self, numCourses, prerequisites):
        """
        :type numCourses: int
        :type prerequisites: List[List[int]]
        :rtype: List[int]
        """

        # Prepare the graph
        adj_list = defaultdict(list)
        indegree = {}
        for dest, src in prerequisites:
            adj_list[src].append(dest)

            # Record each node's in-degree
            indegree[dest] = indegree.get(dest, 0) + 1

        # Queue for maintainig list of nodes that have 0 in-degree
        zero_indegree_queue = deque([k for k in range(numCourses) if k not in indegree])

        topological_sorted_order = []

        # Until there are nodes in the Q
        while zero_indegree_queue:

            # Pop one node with 0 in-degree
            vertex = zero_indegree_queue.popleft()
            topological_sorted_order.append(vertex)

            # Reduce in-degree for all the neighbors
            if vertex in adj_list:
                for neighbor in adj_list[vertex]:
                    indegree[neighbor] -= 1

                    # Add neighbor to Q if in-degree becomes 0
                    if indegree[neighbor] == 0:
                        zero_indegree_queue.append(neighbor)

        return topological_sorted_order if len(topological_sorted_order) == numCourses else []

# DFS - O(v + e) runtime, O(v + e) space where v is the number of courses and e is the number of dependancies

In [6]:
from collections import defaultdict

class Solution:

    WHITE = 1
    GRAY = 2
    BLACK = 3

    def findOrder(self, numCourses, prerequisites):
        """
        :type numCourses: int
        :type prerequisites: List[List[int]]
        :rtype: List[int]
        """

        # Create the adjacency list representation of the graph
        adj_list = defaultdict(list)

        # A pair [a, b] in the input represents edge from b --> a
        for dest, src in prerequisites:
            adj_list[src].append(dest)

        topological_sorted_order = deque()
        is_possible = True

        # By default all vertces are WHITE
        color = {k: Solution.WHITE for k in range(numCourses)}
        def dfs(node):
            nonlocal is_possible

            # Don't recurse further if we found a cycle already
            if not is_possible:
                return

            # Start the recursion
            color[node] = Solution.GRAY

            # Traverse on neighboring vertices
            for neighbor in adj_list[node]:
                if color[neighbor] == Solution.WHITE:
                    dfs(neighbor)
                elif color[neighbor] == Solution.GRAY:
                     # An edge to a GRAY vertex represents a cycle
                    is_possible = False

            # Recursion ends. We mark it as black
            color[node] = Solution.BLACK
            topological_sorted_order.appendleft(node)

        for vertex in range(numCourses):
            # If the node is unprocessed, then call dfs on it.
            if color[vertex] == Solution.WHITE:
                dfs(vertex)

        return topological_sorted_order if is_possible else []

In [8]:
instance = Solution()
instance.findOrder(4, [[1,0],[2,0],[3,1],[3,2]])

[2, 1, 0]